In [145]:
from io import StringIO
import requests
import json
import pandas as pd
import numpy as np
import re, random, collections
import tensorflow as tf
from tensorflow.contrib import rnn

In [146]:
# The code was removed by DSX for sharing.

training_data length: 874
longest sentence: 160
shortest sentence: 1
avrg sentence length: 29.1830663616
standard deviation of sentence length: 17.4226866681
sentence length: 47.0
vocab size: 3214


In [147]:
#fixed vars######################
#sequence length
time_steps=4

#self explanatory
input_size=1

#57 getsure types + no gesture
n_classes=58
no_gesture_index = 57

#changeable vars#####################3
#we need to learn about this
learning_rate=0.001

#could be any number
num_units=10

#for the state array
state_size = num_units*4

#tbd
batch_size=100

#number of LSTM layers
num_layers=2

In [148]:

def create_input_output(flat_data):
    
    #initialize variables
    train_input = []
    train_output = []
    
    single_input = []
    single_output = [0]*n_classes    
    
    #'''for every word in the data set'''
    for i in range(len(flat_data)):
    
        count = 0
        extra_index = 0
        
        #'''get them in groups of four'''
        words = flat_data[i+extra_index:i+time_steps+extra_index]
        
        #'''if we have reached the end of the dataset end this function'''
        if len(words) != time_steps:
            break
        
        #'''until the single_output is full'''
        while len(single_input) < time_steps:
            
            word = words[count] 
            gesture = re.search(r'g\d{1,2}', word)

            #'''if the word being analysed is not a gesture'''
            if gesture is None:

                #'''find correspondig value in dictionary and add to input'''
                num = dictionary[word]
                single_input.append(num)
                

            #'''if the word being analysed is a gesture'''
            elif gesture is not None:
            
                
                #'''get the next word from the flatlist and add it to the word to be analysed array'''
                new_word = flat_data[i+time_steps+extra_index]
                words.append(new_word)
                extra_index = extra_index + 1
               
            
            #'''when the input array is full'''
            if len(single_input) == time_steps:
                
                gesture_check = None
                
                #'''get the next entry in the flat list and check if it's a gesture'''
                if i+time_steps+extra_index < len(flat_data):
                    next_entry = flat_data[i+time_steps+extra_index]
                    gesture_check = re.search(r'g\d{1,2}', next_entry)
                
                #'''if yes, mark that gesture in the output of this particular input'''
                if gesture_check is not None:
                    index_gesture = int(next_entry[1:])
                    single_output[index_gesture] = 1
                    
                #'''if no, mark the no gesture cell in the output array'''
                else:
                    single_output[no_gesture_index] = 1
                    
                
            count = count+1
                
            
        train_input.append(single_input)
        train_output.append(single_output)  

        single_input = []
        single_output = [0]*n_classes  
    
    return train_input, train_output
            
#'''make list from list of lists'''
flat_training_data = [item for sublist in training_data for item in sublist]

#'''populate the input and output data'''
train_input, train_output = create_input_output(flat_training_data)
                       

#creating the weight array for the weighted loss function#########################################
counts = [item.index(1) for item in train_output]
count_total = [counts.count(i) for i in range(0,58)]
print(count_total)

weight_array_1 = [1 - (item/sum(count_total)) for item in count_total]
weight_array_2 = [weight/sum(weight_array_1) for weight in weight_array_1]



#Random sampling training and test data###################################################
_TRAINING_INS_COUNT = 20500
train_indexes = random.sample(range(0, len(train_input)), _TRAINING_INS_COUNT)
test_indexes = [i for i in range(0, len(train_input)) if not i in train_indexes]

#creating training data 80%
_TRAINING_INPUT = np.array(train_input)
_TRAINING_INPUT = _TRAINING_INPUT[train_indexes]
_TRAINING_OUTPUT = np.array(train_output)
_TRAINING_OUTPUT = _TRAINING_OUTPUT[train_indexes]

#creating testing data 20%
_TESTING_INPUT = np.array(train_input)
_TESTING_INPUT = _TESTING_INPUT[test_indexes]
print(_TESTING_INPUT)
_TESTING_OUTPUT = np.array(train_output)
_TESTING_OUTPUT = _TESTING_OUTPUT[test_indexes]


[30, 59, 27, 17, 94, 55, 27, 63, 97, 95, 14, 26, 45, 40, 27, 41, 34, 30, 61, 16, 49, 71, 46, 50, 42, 101, 21, 20, 37, 34, 7, 38, 18, 35, 99, 109, 23, 52, 27, 15, 12, 13, 17, 19, 28, 21, 14, 31, 15, 25, 22, 10, 19, 24, 123, 6, 19, 23293]
[[ 925   91  275   57]
 [  91  275   57  757]
 [  57  757    4   56]
 ..., 
 [  67    4  238   90]
 [1717   11   51  259]
 [  51  259    1    1]]


In [149]:
tf.reset_default_graph()

#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]), name="weights")
out_bias=tf.Variable(tf.random_normal([n_classes]), name="biases")

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,input_size])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])
print(x)

#processing the input tensor from [batch_size,n_steps,input_size] to "time_steps" number of [batch_size,input_size] tensors
input=tf.unstack(x ,time_steps,1)
print(input)

Tensor("Placeholder:0", shape=(?, 4, 1), dtype=float32)
[<tf.Tensor 'unstack:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'unstack:1' shape=(?, 1) dtype=float32>, <tf.Tensor 'unstack:2' shape=(?, 1) dtype=float32>, <tf.Tensor 'unstack:3' shape=(?, 1) dtype=float32>]


In [150]:
def lstm_cell():
  return tf.contrib.rnn.BasicLSTMCell(num_units)
stacked_lstm = tf.contrib.rnn.MultiRNNCell(
    [lstm_cell() for _ in range(num_layers)])


In [151]:
initial_state = state = stacked_lstm.zero_state(batch_size, tf.float32)


In [152]:
with tf.variable_scope("rnn", reuse=None):
    outputs,state=tf.nn.dynamic_rnn(stacked_lstm,x,dtype='float32')
    print(state)
    
    pred_ready_output = tf.unstack(outputs,time_steps,1)
        
    #converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
    prediction=tf.matmul(pred_ready_output[-1],out_weights)+out_bias

    #loss_function
    loss=tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(logits=prediction,targets=y,pos_weight=tf.constant(weight_array_1)))
    #optimization
    opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

    #model evaluationmatmul
    correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

(LSTMStateTuple(c=<tf.Tensor 'rnn/rnn/while/Exit_2:0' shape=(?, 10) dtype=float32>, h=<tf.Tensor 'rnn/rnn/while/Exit_3:0' shape=(?, 10) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'rnn/rnn/while/Exit_4:0' shape=(?, 10) dtype=float32>, h=<tf.Tensor 'rnn/rnn/while/Exit_5:0' shape=(?, 10) dtype=float32>))


In [153]:
#initialize variables
batch_init = 0
limit = batch_init + batch_size
init=tf.global_variables_initializer()

with tf.Session() as sess:
    
    sess.run(init)
    iter=1
    while iter<205:
        
        batch_x = _TRAINING_INPUT[batch_init: limit]
        batch_y = _TRAINING_OUTPUT[batch_init: limit]
        batch_init = batch_init + batch_size
        limit = batch_init + batch_size

        batch_x=batch_x.reshape((batch_size,time_steps,input_size))

        sess.run(opt, feed_dict={x: batch_x, y: batch_y})
        
        raw_predictions = prediction.eval(feed_dict={x: batch_x}, session=sess).tolist()
        print(len(raw_predictions))
        results = [sublist.index(max(sublist)) for sublist in raw_predictions]
        print(results)

        if iter %10==0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1        

    #calculating test accuracy
    #test_data = _TESTING_INPUT.reshape((-1, time_steps, input_size))
    #test_label = _TESTING_OUTPUT
    #print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))
    
    #print("Prediction:", sess.run(prediction, feed_dict={x: test_data, y: test_label}))
    #d = prediction.eval(feed_dict={x: test_data}, session=sess)
    #print ("predictions", d)
    #print(type(d[0]))

100
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
100
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
100
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
100
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,